In [1]:
import sys
print(sys.path)
# sys.path = []
sys.path.remove('/home/ayan/catkin_ws/devel/lib/python2.7/dist-packages')
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
# sys.path.append('/usr/bin/python3.5/dist-packages')
# sys.path.append('/usr/bin/python2.7/dist-packages')
# sys.path.append('/home/ayan/.local/lib/python2.7/site-packages')
    # sys.path.append('/usr/local/lib/python2.7/dist-packages')
sys.path.append('/usr/local/lib/python3.5/dist-packages')

import cv2 as cv
import imutils 
import numpy as np
import time
import serial


['/home/ayan/fun_projects/prof_cam', '', '/home/ayan/catkin_ws/devel/lib/python2.7/dist-packages', '/opt/ros/kinetic/lib/python2.7/dist-packages', '/home/ayan/strands_qsr_lib/qsr_lib/src', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/ayan/.local/lib/python2.7/site-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/PILcompat', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/lib/python2.7/dist-packages/wx-3.0-gtk2', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/home/ayan/.ipython']


In [2]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv.resize(frame, dim, interpolation =cv.INTER_AREA)

In [3]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

inWidth = 256
inHeight = 144

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

cap = cv.VideoCapture(0)

flag_out = False
flag_in = False

ser = serial.Serial(
    port='/dev/ttyACM1',
    baudrate=9600,
    timeout=1
)

print(ser.isOpen())


in_count = 0
out_count = 0

while cv.waitKey(1) < 0:
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frame = rescale_frame(frame, percent=150)

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > 0.05 else None)

    direction = ''

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

        if points[idFrom] and points[idTo]:
            if points[idFrom][0] < 120 or points[idTo][0] < 120:
                out_count+=1 
                direction = 'right'
                # print('it should be turning left')
            elif points[idFrom][0] > 800 or points[idTo][0] > 800:
                out_count+=1
                direction = 'left'
                # print('it should be turning right')
            else:
                in_count+=1
    
    if out_count > in_count:
        print('OUT of frame')
        flag_out = True
        flag_in = False
        ser.write(direction + '\n')
        out = ''
        time.sleep(0.1)
        # let's wait one second before reading output (let's give device time to answer)
        # time.sleep(1)
        # while ser.inWaiting() > 0:
        #     out += ser.read(1)

        # if out != '':
        #     print(">>" + out)

        # if out == '':
        #     print("wtaf")
    
    if in_count > out_count:
        # print('In Frame')
        flag_in = True
        flag_out = False 
        out_count = 0
        in_count = 1
        ser.write('stay\n')


    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    
    # frame = rescale_frame(frame, percent=150)

    cv.imshow('frame150', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

True
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of frame
OUT of 

In [4]:

# # configure the serial connections (the parameters differs on the device you are connecting to)
# ser = serial.Serial(
#     port='/dev/ttyACM0',
#     baudrate=9600,
#     parity=serial.PARITY_ODD,
#     stopbits=serial.STOPBITS_TWO,
#     bytesize=serial.SEVENBITS
# )

# ser.isOpen()

# print 'Enter your commands below.\r\nInsert "exit" to leave the application.'

# input=1
# while 1 :
#     # get keyboard input
#     input = raw_input(">> ")
#         # Python 3 users
#         # input = input(">> ")
#     if input == 'exit':
#         ser.close()
#         exit()
#     else:
#         # send the character to the device
#         # (note that I happend a \r\n carriage return and line feed to the characters - this is requested by my device)
#         ser.write(input + '\r\n')
#         out = ''
#         # let's wait one second before reading output (let's give device time to answer)
#         time.sleep(1)
#         while ser.inWaiting() > 0:
#             out += ser.read(1)

#         if out != '':
#             print ">>" + out

In [5]:

# # Initializing the HOG person 
# # detector 
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
# cap = cv2.VideoCapture('output_random2.mp4')
# # cap = cv2.VideoCapture(0)

# while 1:
#     ret, img = cap.read()
#     # img = imutils.resize(img, width=min(100, img.shape[1]))
#     img = imutils.resize(img, width=1000)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.02, 5, minSize = (50, 80))

#     for (x,y,w,h) in faces:
#         cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
#         # roi_gray = gray[y:y+h, x:x+w]
#         # roi_color = img[y:y+h, x:x+w]
        
#     cv2.imshow('img',img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
    

# cap.release()
# cv2.destroyAllWindows()
